In [ ]:
!gdown 12ZlsMNXutxN3U_V-IvLGQYpPT9DopGjM -O sample-data.zip

In [ ]:
!unzip sample-data.zip -d ./data/original

In [5]:
import os
import glob
import cv2
import numpy as np

In [6]:
def gammaCorrection(src, gamma):
  invGamma = 1/gamma
  table = [((i / 255) ** invGamma) * 255 for i in range(256)]
  table = np.array(table, np.uint8)

  return cv2.LUT(src, table)

In [ ]:
input_base_path = "./data/original"
output_base_path = "./data/enhanced"

clahe_clip = 2
gamma = 5
tophat_kernel_size = (3, 3)

for directory in os.listdir(input_base_path):
  input_path = os.path.join(input_base_path, directory)
  output_path = os.path.join(output_base_path, directory)

  if not os.path.exists(output_path):
    os.makedirs(output_path, exist_ok=True)
    
  for filename in glob.glob(os.path.join(input_path,"*.png")):
    print(output_path, filename)
    img = cv2.imread(filename)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    hue, sat, val = cv2.split(hsv)

    inv_val = 255-val
    clahe = cv2.createCLAHE(clipLimit = clahe_clip)
    clahe_res = clahe.apply(inv_val)

    gamma_res = gammaCorrection(clahe_res, 1/gamma)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, tophat_kernel_size)
    tophat = cv2.morphologyEx(gamma_res, cv2.MORPH_TOPHAT, kernel)

    tophat_res = cv2.add(gamma_res, tophat)

    arr1 = clahe_res.reshape(-1,1)
    arr2 = tophat_res.reshape(-1,1)

    matrix = np.concatenate((arr1, arr2), axis=1)
    mean, eigenvectors = cv2.PCACompute(matrix, mean=None)

    pc1 = eigenvectors[0,:]
    w1 = pc1[0]/(pc1[0]+pc1[1])
    w2 = pc1[1]/(pc1[0]+pc1[1])

    fusion = (w2*clahe_res+w1*tophat_res).astype(np.uint8)

    final_inv = 255-fusion

    merged = cv2.merge([hue, sat, final_inv])
    merged_rgb = cv2.cvtColor(merged, cv2.COLOR_HSV2BGR)
    cv2.imwrite(os.path.join(output_path,
                             os.path.basename(filename)),
                             merged_rgb)